In [2]:
!python -m pip install mlflow yfinance langchain langchain-community newsapi-python yfinance--quiet


ERROR: Could not find a version that satisfies the requirement yfinance--quiet (from versions: none)
ERROR: No matching distribution found for yfinance--quiet


In [4]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [5]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000/")
print(mlflow.__version__)

3.3.2


In [6]:
from pydantic import BaseModel
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

class Stock(BaseModel):
    name: str
    stock_code: str

class StockResponse(BaseModel):
    companies: list[Stock]

parser = PydanticOutputParser(pydantic_object=StockResponse)


In [7]:
from langchain_openai import AzureChatOpenAI
model_name='gpt4o'
model = AzureChatOpenAI(model=model_name)


In [8]:
# import mlflow
# from functools import wraps

# def mlflow_log(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         with mlflow.start_run(run_name=f"{func.__name__}_prateekagashe", nested=True):
#             mlflow.log_text(str(args), artifact_file="args.txt")
#             mlflow.log_text(str(kwargs), artifact_file="kwargs.txt")

#             result = func(*args, **kwargs)

#             mlflow.log_text(str(result), artifact_file="result.txt")
#             return result
#     return wrapper


def mlflow_log(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        mlflow.log_text(str(result), f"{func.__name__}_output.txt")
        return result
    return wrapper


In [7]:
import mlflow

initial_template = """\
Generate stock code for the following company(s) {{ company_name }}.

Return JSON with stock codes. Include exchange suffix if not US stock. 
Example:

{
  "companies": [
    {"name": "Microsoft", "stock_code": "MSFT"},
    {"name": "Apple", "stock_code": "AAPL"},
    {"name": "Tesla", "stock_code": "TSLA"},
    {"name": "TCS", "stock_code": "TCS.NS"} # example for outside US stock
  ]
}



Do not use any other keys, do not include explanations or text outside JSON.
"""

prompt = mlflow.genai.register_prompt(
    name="prateekagashe-stock-code-prompt",
    template=initial_template,
    commit_message="Initial commit",
    tags={
        "author": "Prateek Agashe",
        "task": "Stock Coge Generation",
        "language": "en",
        'llm': 'gpt-4o-mini'
    },
)

print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/09/21 16:15:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: prateekagashe-stock-code-prompt, version 6


Created prompt 'prateekagashe-stock-code-prompt' (version 6)


In [50]:
import mlflow

initial_template = """\
Analyze the following news article for the company {{ company_name }} ({{ stock_code }}).

Return a JSON object with the following fields:
- company_name: string (always return exactly as provided)
- stock_code: string (always return exactly as provided)
- newsdesc: string (the full news article text exactly as provided)
- sentiment: one of ["Positive", "Negative", "Neutral"]
- people_names: list of people mentioned
- places_names: list of places mentioned
- other_companies_referred: list of other companies mentioned
- related_industries: list of related industries
- market_implications: a short explanation of how this news could affect the market perception or stock
- confidence_score: float between 0 and 1

News Article:
{{ news_text }}

Return a JSON object following these format instructions:

{{format_instructions}}

Do not include explanations, commentary, or any keys not listed above.
"""


prompt = mlflow.genai.register_prompt(
    name="prateekagashe-sentiment-analyzer-prompt",
    template=initial_template,
    commit_message="Initial commit",
    tags={
        "author": "Prateek Agashe",
        "task": "Stock sentiment Generation",
        "language": "en",
        'llm': 'gpt-4o-mini'
    },
)

print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/09/21 18:48:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: prateekagashe-sentiment-analyzer-prompt, version 9


Created prompt 'prateekagashe-sentiment-analyzer-prompt' (version 9)


In [51]:
# mlflow.set_prompt_alias("prateekagashe-stock-code-prompt", alias="default", version=5)
mlflow.set_prompt_alias("prateekagashe-sentiment-analyzer-prompt", alias="default", version=9)

/tmp/ipykernel_2984/3420911521.py:2: FutureWarning: The `mlflow.set_prompt_alias` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.set_prompt_alias` instead. The original API will be removed in the future release.
  mlflow.set_prompt_alias("prateekagashe-sentiment-analyzer-prompt", alias="default", version=9)


In [9]:
import mlflow


def stock_code_prompt(companies: list[str]) -> str:
    stock_code_prompt_template = mlflow.load_prompt(f"prompts:/prateekagashe-stock-code-prompt@default")

    return stock_code_prompt_template.format(company_name=", ".join(companies))

In [10]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


response_schemas = [
    ResponseSchema(name="company_name", description="Name of the company"),
    ResponseSchema(name="stock_code", description="Stock ticker code"),
    ResponseSchema(name="newsdesc", description="Full news article text"),
    ResponseSchema(name="sentiment", description='One of "Positive", "Negative", "Neutral"'),
    ResponseSchema(name="people_names", description="List of people mentioned"),
    ResponseSchema(name="places_names", description="List of places mentioned"),
    ResponseSchema(name="other_companies_referred", description="List of other companies mentioned"),
    ResponseSchema(name="related_industries", description="List of related industries"),
    ResponseSchema(name="market_implications", description="Short explanation of market impact"),
    ResponseSchema(name="confidence_score", description="Float between 0 and 1 indicating confidence"),
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()


In [11]:
import mlflow

template = mlflow.load_prompt(f"prompts:/prateekagashe-sentiment-analyzer-prompt@default")

def stock_sentiment_prompt(company_name: str, stock_code: str, news_text: str):

    full_prompt = template.format(
        company_name=company_name,
        stock_code=stock_code,
        news_text=news_text,
        format_instructions=format_instructions
    )
    return full_prompt


/tmp/ipykernel_7092/3104543603.py:3: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  template = mlflow.load_prompt(f"prompts:/prateekagashe-sentiment-analyzer-prompt@default")


In [12]:

@mlflow_log
def generateStockCode(prompt):

    response  = model.invoke(prompt)
    return response.content



In [13]:
@mlflow_log
def extract_news_text(articles, company_name):

    relevant_articles = [
        a for a in articles
        if company_name.lower() in (a.get("title","") + a.get("description","")).lower()
    ]
    
    news_texts = []
    for a in relevant_articles:
        title = a.get("title", "")
        description = a.get("description", "")
        url = a.get("url", "")
        news_texts.append(f"{title} - {description} ({url})")
    
    return "\n\n".join(news_texts[:5])

In [ ]:
import yfinance as yf
import json
import requests

POLYGON_API_KEY = "XXXXXXXXXXXXXXXXXXXXX"
@mlflow_log
def generateStockNews(stock_data, news_limit: int = 10):
    news_data = {"companies": []}
    
    if isinstance(stock_data, str):
        stock_data = json.loads(stock_data)

    for company in stock_data["companies"]:
        name = company["name"]
        stock_code = company["stock_code"]
        print(f"Fetching data for {name} ({stock_code})")
        
        try:
            stock = yf.Ticker(stock_code)
            hist = stock.history(period="1d")
            price = hist["Close"].iloc[-1] if not hist.empty else None
        except Exception as e:
            print(f"Error fetching price for {stock_code}: {e}")
            price = None
        
        news_text = ""
        try:
            url = f"https://api.polygon.io/v2/reference/news?ticker={stock_code}&limit={news_limit}&apiKey={POLYGON_API_KEY}"
            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                articles = []
                for item in data.get("results", []):
                    title = item.get("title", "")
                    description = item.get("description", "")
                    url = item.get("url", "")
                    articles.append(f"{title} - {description} ({url})")
                news_text = "\n\n".join(articles)
            else:
                print(f"Error fetching news for {stock_code}: {response.text}")
        except Exception as e:
            print(f"Exception fetching news for {stock_code}: {e}")
        
        news_data["companies"].append({
            "name": name,
            "stock_code": stock_code,
            "latest_price": price,
            "news": news_text
        })

    return news_data


In [15]:
@mlflow_log
def analyze_sentiment(companies_data):
    structured_results = []

    for company in companies_data["companies"]:
        prompt = stock_sentiment_prompt(
            company_name=company["name"],
            stock_code=company["stock_code"],
            news_text=company["news"]
        )

        sentiment_response = model.invoke(prompt)
        result  = parser.parse(sentiment_response.content)
        structured_results.append(result)
    return structured_results


In [18]:
from langchain.schema.runnable import RunnableLambda
import mlflow
stock_prompt_runnable = RunnableLambda(lambda companies: stock_code_prompt(companies))
stock_code_runnable = RunnableLambda(lambda prompt: generateStockCode(prompt))
stock_news_runnable = RunnableLambda(lambda stock_codes: generateStockNews(stock_codes))
stock_sentiment_runnable = RunnableLambda(lambda stock_news: analyze_sentiment(stock_news))


chain = stock_prompt_runnable | stock_code_runnable | stock_news_runnable | stock_sentiment_runnable

companies_list = ["Microsoft", "Apple Inc"]
mlflow.set_experiment("StockSentimentAnalysis_PrateekAgashe")

with mlflow.start_run(run_name="StockSentimentChain_prateekagashe"):
    result = chain.invoke(companies_list)
    mlflow.log_text(str(result), "final_result.json")
    print(result)



2025/09/21 20:04:36 INFO mlflow.tracking.fluent: Experiment with name 'StockSentimentAnalysis_PrateekAgashe' does not exist. Creating a new experiment.
/tmp/ipykernel_7092/3103021693.py:5: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  stock_code_prompt_template = mlflow.load_prompt(f"prompts:/prateekagashe-stock-code-prompt@default")


Fetching data for Microsoft (MSFT)
Fetching data for Apple Inc. (AAPL)
[{'company_name': 'Microsoft', 'stock_code': 'MSFT', 'newsdesc': 'Nvidia, Microsoft Sit Atop A $23.9 Trillion Empire—More Than China And Germany Combined - Top 10 U.S. stocks have reached a combined market capitalization of $23.9 trillion, surpassing the GDPs of China and Germany. Nvidia leads with a $4.25 trillion market cap, driven by the AI boom, with seven tech giants responsible for over $20 trillion of total value.', 'sentiment': 'Positive', 'people_names': [], 'places_names': ['China', 'Germany'], 'other_companies_referred': ['Nvidia'], 'related_industries': ['Technology', 'AI', 'Finance'], 'market_implications': 'This news showcases the strong market position of Microsoft and its peers, indicating confidence in their growth potential, especially in the AI sector, which may positively influence Microsoft’s stock performance.', 'confidence_score': 0.8}, {'company_name': 'Apple Inc.', 'stock_code': 'AAPL', 'new